In [1]:
# PREVIOUS VERSION WILL ACQUIRE TWEETS AND PUT THEM IN SQL DATABASE TWEETS
# PREVIOUS VERSION WILL ACQUIRE OLD TWEETS THAT HAVE BEEN RETWEETED
# THIS VERSION WILL NOT COMMITT ANY RETWEETS OLDER THAN 7 Days
# THIS VERSION WILL BREAK UP TWEETS INTO AN EQUAL NUMBER OF REQUESTS FOR EACH DAY

In [2]:
#Twitter apikey - VnEUFK7HUgAwcSa4yNbATnGrc
#Twitter api secretkey - Tl0ywm860EWFSIC3dM4HrZ8Fw92kTrsFJOylUmfTdEatYAZm0k
#Twitter bearer token - AAAAAAAAAAAAAAAAAAAAAJt4HAEAAAAA%2BuMkCsqy%2FE3mXODKjKItDO%2B8IoY%3Ds1FYI4Q0I23yWyMoiVPPUNctfZDpNljdAA8P5HcI37vq2tqhyW
#On 01/12/2021 your consumer keys will no longer be visible.

consumer_key = "VnEUFK7HUgAwcSa4yNbATnGrc"
consumer_secret = "Tl0ywm860EWFSIC3dM4HrZ8Fw92kTrsFJOylUmfTdEatYAZm0k"
token = "AAAAAAAAAAAAAAAAAAAAAJt4HAEAAAAA%2BuMkCsqy%2FE3mXODKjKItDO%2B8IoY%3Ds1FYI4Q0I23yWyMoiVPPUNctfZDpNljdAA8P5HcI37vq2tqhyW"

access_token = '1298664925217210370-wRovRCyte8JShjsjg3k2MtBmMMe9na'
access_token_secret = 'd8Ibm3dVSRMgXPkGCaEA7QbBf8IuY8cWGUdI65Au3I5g6'


In [3]:
#pip install tweepy
#pip install textblob
import os
import tweepy as tw
import pandas as pd
import json
import sqlite3
from textblob import TextBlob
import re
from datetime import datetime as dt, timedelta

#access to API
auth = tw.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)


In [4]:
#Below functions are meant to be combined in one main function

In [5]:
#Accesses Twitter based on user inputed search criteria - returns dictionary with 2 items 
# 1: List of 7 search results 
# 2: Search term

def get_tweets():
    #inputs for what you want to search and how many results to get
    search_words = input("Input Search Here:")
    
    num_results = input("Sets of 100 Tweets Desired (Press enter for maximum):")
    if num_results == '':
        num_results = 180 #maximum pages for rate limit
    elif int(num_results) < 7:
        num_results = 7 #has to be at least 7 - one request per day
    else:
        num_results = int(num_results)
    num_results = round(num_results/7) #requests per date
    
    date_list = []
    current_date = dt.utcnow() 
    #Gives list of dates in format for the API
    for i in range(7):
        new_date = (current_date - timedelta(days=i)).strftime("%Y-%m-%d")
        date_list.append(new_date)
        
    tweets = []
    for date in date_list:
        day_tweets = tw.Cursor(api.search, q = search_words, lang = "en", result_type = 'mixed', count = 100 ,until = date, tweet_mode = 'extended').pages(num_results)
        tweets.append(day_tweets)
        
    return {'tweets':tweets, 'search_words':search_words}


In [6]:
#Takes retrieved tweets and creates smaller list
def create_tweet_list(tweepy_cursor):
    #Create a list with only the items we want from the tweet - decrease storage
    #Dict keys - #['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang']
    #add any of above keys if data is useful
    desired_keys = ['created_at', 'id', 'full_text', 'entities'
                    , 'metadata', 'in_reply_to_status_id'
                    ,'in_reply_to_user_id', 'user', 'place', 'retweeted_status'
                    , 'retweet_count', 'favorite_count', 'retweeted']
    tweet_list = []
    for page in tweepy_cursor:
        for status in page:
            status = status._json #gets the dictionary object only
            newdict = {} 
            for key in desired_keys: #iterate through list of the keys we want
                try:
                    newdict.update({ key : status[key] }) #adds new key:value pair to dictionary item for a status
                except KeyError: #loop back to key loop if there isn't a key present - used when there isnt a retweeted_status key
                    continue
            tweet_list.append(newdict)
    return tweet_list

In [7]:
def compile_tweets():
    pull_tweets = get_tweets() 
    search_results = pull_tweets['tweets'] #this returns the list of 7 daily search results
    result_list = []
    #creates a list of lists with 7 elements
    for daily_search_results in search_results:
        new_results = create_tweet_list(daily_search_results) #this applies a function to refine that full list of tweets
        result_list.append(new_results)
    #flatten the list
    big_list = []
    for sublist in result_list:
        for item in sublist:
            big_list.append(item)
    search_words = pull_tweets['search_words'] #returns the search term used to acquire the tweets
    return {'tweets':big_list, 'search_words':search_words}


In [8]:
#Takes the list of tweets returned from the api search and adds them to the SQL "Tweets2" database

def add_to_DB():
    tweet_data = compile_tweets() #calls function to get list of tweets and the search term used
    list_of_tweets = tweet_data['tweets']
    search_words = tweet_data['search_words']
    conn = sqlite3.connect('tweets2.sqlite')
    cur = conn.cursor()

    #Delete table if needed - only for when I'm testing stuff
    cur.execute('''DROP TABLE IF EXISTS Tweets2''')

    #Create the table and define data parameters
    #Tweet ID is the primary key because we are interested in specific tweets
    cur.execute('''CREATE TABLE IF NOT EXISTS Tweets2
                    (tweet_id INTEGER PRIMARY KEY UNIQUE
                    , user_id INTEGER
                    , user_name TEXT, name TEXT
                    , text TEXT , time_posted INTEGER
                    , retweet_count INTEGER, favorite_count INTEGER
                    , place_id TEXT, place_name TEXT, place_coord TEXT
                    , search_words TEXT, sentiment REAL, subjectivity REAL, clean_text TEXT, clean_time INTEGER)''')


    #Create values to input into SQL table
    error_tweets = [] #list of dictionaries to hold all items where errors occur
    count = 0
    for tweet in list_of_tweets:
        count += 1
        #First - If status is a retweet, we want to ignore it
        try:
            tweet['retweeted_status']
            continue #Want to restart loop if we run into a retweet
        #If it is not a retweet, then will give KeyError, so use 
        except KeyError:
            tweet_id = tweet['id']
            user_id = tweet['user']['id']
            user_name = tweet['user']['screen_name']
            name = tweet['user']['name']
            text = tweet['full_text']
            time_posted = tweet['created_at']
            retweets = tweet['retweet_count']
            favorites = tweet['favorite_count']
            try:
                place_id = tweet['place']['id']
            except:
                place_id = None
            try:
                place_name = tweet['place']['full_name']
            except:
                place_name = None
            try:
                place_coord = tweet['place']['coordinates']
            except:
                place_coord = None
        
        #clean the text & re-format date to unix timestamp for sql
        clean_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 
        clean_time = dt.strptime(time_posted, '%a %b %d %H:%M:%S %z %Y').timestamp()

        #check polarity
        sentiment = TextBlob(clean_text).sentiment[0] # (-1) to 1 scale
        subjectivity =  TextBlob(clean_text).sentiment[1] # 0-1 scale
        
        #Insert into SQL table - Tweets
        try:
            cur.execute('''INSERT OR IGNORE INTO Tweets2
            (tweet_id, user_id, user_name, name, text, time_posted, retweet_count
            , favorite_count, place_id, place_name, place_coord, search_words, sentiment, subjectivity, clean_text, clean_time) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
            (tweet_id, user_id, user_name, name, text, time_posted, retweets, favorites
            , place_id, place_name, place_coord, search_words, sentiment, subjectivity, clean_text, clean_time) )

        except: 
            print("error", count)
            error_dict = {"tweet_id":tweet_id, "user_id":user_id, "user_name":user_name, "name":name, "text":text, "time_posted":time_posted, "retweet_count":retweets, "favorite_count":favorites, "place_id":place_id, "place_name":place_name, "place_coord":place_coord}
            error_tweets.append(error_dict)
            pass

        #commit every 100th OR when we have reach the end of the list of tweets
        if ((count % 1000 == 0) or (count == len(list_of_tweets)-1)) and count > 0:
            conn.commit()
            print("Committed", count)
            #Notify that all have been committed
            if count == len(list_of_tweets)-1:
                cur.execute('''SELECT COUNT(tweet_id) FROM Tweets2''')
                num_tweets = cur.fetchone()[0] #if I dont do 0 it returns a one item list
                print('Finished - ', num_tweets, "tweets added to database out of", len(list_of_tweets), "tweets" )
          
    cur.close() #always close the connection   


In [10]:
#add_to_DB()

In [12]:
#Gives the sentiment for each day
def graph_data():
    conn = sqlite3.connect('tweets2.sqlite')
    cur = conn.cursor()
    cur.execute('''SELECT SUM(sentiment * (CASE WHEN retweet_count > 0 THEN retweet_count ELSE 1 END))
                        /(SUM(retweet_count) + COUNT(tweet_id)) AS Sentiment
                    , time_posted as Date
                    , COUNT(tweet_id) as Num_Tweets
                FROM Tweets2 
                GROUP BY STRFTIME('%d', datetime(clean_time, 'unixepoch'))''') 
                #get the average sentiment of tweets that have a sentiment rating calculated

    q = cur.fetchall()

    #Create a list of results for the graph
    daily_data = []
    for item in q:
        daily_list = []
        date = item[1][0:10]
        sentiment = round(item[0], 2)
        daily_list = [date, sentiment]
        daily_data.append(daily_list)
    
    cur.close()
    return daily_data

#Gives tweets positively affecting sentiment the most each day
def pos_influencers():
    conn = sqlite3.connect('tweets2.sqlite')
    cur = conn.cursor()
    
    cur.execute('''SELECT MAX( sentiment * (1 + retweet_count) ), time_posted, retweet_count, sentiment, name, text
                    FROM Tweets2
                    GROUP BY strftime('%d', datetime(clean_time, 'unixepoch'))''')
    
    q2 = cur.fetchall()
    
    pos_influencers = []
    for item in q2:
        date = item[1][0:10]
        retweets = item[2]
        sentiment = round(item[3], 2)
        name = item[4]
        text = item[5]
        new_list = [date, name, text, sentiment, retweets]
        pos_influencers.append(new_list)  
     
    #Gives tweets negatively affecting sentiment the most each day

    cur.close()
    return pos_influencers
    
def neg_influencers():
    conn = sqlite3.connect('tweets2.sqlite')
    cur = conn.cursor()    
    
    cur.execute('''SELECT MIN( sentiment * (1 + retweet_count) ), time_posted, retweet_count, sentiment, name, text
                FROM Tweets2
                GROUP BY strftime('%d', datetime(clean_time, 'unixepoch'))''')    

    q3 = cur.fetchall()
    
    neg_influencers = []
    for item in q3:
        date = item[1][0:10]
        retweets = item[2]
        sentiment = round(item[3], 2)
        name = item[4]
        text = item[5]
        new_list = [date, name, text, sentiment, retweets]
        neg_influencers.append(new_list) 
        
    cur.close()
    return neg_influencers


In [16]:
#next is to simply graph the results and print a nice list of the top influencers
def print_influencers():
    pos = pos_influencers()
    neg = neg_influencers()
    print("Most Influential Positive Tweets:", '\n', '----------------------')
    count = 1
    for i in pos:
        print(count, '.', i[0], '-', i[1], '-', i[2] )
        print('Sentiment:', i[3], '-', 'Retweets:', i[4])
        count += 1
        print('\n')
    
    print('----------------------')
    
    print("Most Influential Negative Tweets:", '\n', '----------------------')
    count = 1
    for i in neg:
        print(count, '.', i[0], '-', i[1], '-', i[2] )
        print('Sentiment:', i[3], '-', 'Retweets:', i[4])
        count += 1
        print('\n')
        

In [18]:
def get_top_influencers():
    add_to_DB()
    print('\n')
    print_influencers()

get_top_influencers()

Input Search Here: Tour de France
Sets of 100 Tweets Desired (Press enter for maximum): 15


Committed 1392
Finished -  611 tweets added to database out of 1393 tweets


Most Influential Positive Tweets: 
 ----------------------
1 . Wed Sep 02 - ITV Cycling - Sam Bennett becomes the first Irish rider in more than three decades to lead a Tour de France classification! @Sammmy_Be 👏💚

#TDF2020 https://t.co/5EAbtYXFlo
Sentiment: 0.25 - Retweets: 115


2 . Fri Sep 04 - Peter Sagan - What an amazing job today by all my @BORAhansgrohe teammates, from start to finish. It's a pity that a mechanical in the sprint didn't allow me to finish off this huge effort. This is part of cycling... @BORAhansgrohe  https://t.co/DfXGB35yfh
Sentiment: 0.3 - Retweets: 242


3 . Sat Sep 05 - #Wine Guru🍷 - #Wine #Food: Tour de France by the Glass 2020 Stage 9: Bicycles, Bottles, and Books Before we look ahead to Sunday’s stage, let’s revisit how Stage 8 turned out. It was a good day for the French, with Nans Peters of the AG2R racing... https://t.co/oRiTfIbuQV via @theswirlingderv https://t.co/oPSXYUu2my